In [4]:
from openai import OpenAI
import json

BASE_URL = "http://0.0.0.0:8080/v1"
API_KEY = "sk-xxx"
MODEL_ID = "functionary-7b-v2.1"

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

# Baseline

Test out a question without using tools as a baseline

In [5]:
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {
            "content": "You are a helpful assistant.",
            "role": "system"
        },
        {
            "content": "What is the capital of France?",
            "role": "user"
        }
    ],
)
for choice in response.choices:
    print(choice.message.content)

The capital of France is Paris.


In [6]:
def chat_completion_request(messages, tools=None, model=MODEL_ID):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            # tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [7]:
messages = [
    {"role": "user", "content": "what's the weather like in Hanoi?"}
]
tools = [ # For functionary-7b-v2 we use "tools"; for functionary-7b-v1.4 we use "functions" = [{"name": "get_current_weather", "description":..., "parameters": ....}]
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g., San Francisco, CA",
                    }
                },
                "required": ["location"]
            }
        }
    }
]

In [8]:
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response

ChatCompletion(id='chatcmpl-d08d6502-cd60-42dd-b0f0-766fda8b4736', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xuFI2ULkH5W6GZRy1MjHSICx', function=Function(arguments='{"location": "Hanoi, Vietnam"}', name='get_current_weather'), type='function')]))], created=1720034000, model='functionary-7b-v2.1', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=163, total_tokens=164))

In [11]:
import yaml

print(yaml.dump(chat_response.model_dump(), sort_keys=False, explicit_start=True))


---
id: chatcmpl-d08d6502-cd60-42dd-b0f0-766fda8b4736
choices:
- finish_reason: tool_calls
  index: 0
  logprobs: null
  message:
    content: null
    role: assistant
    function_call: null
    tool_calls:
    - id: call_xuFI2ULkH5W6GZRy1MjHSICx
      function:
        arguments: '{"location": "Hanoi, Vietnam"}'
        name: get_current_weather
      type: function
created: 1720034000
model: functionary-7b-v2.1
object: chat.completion
service_tier: null
system_fingerprint: null
usage:
  completion_tokens: 20
  prompt_tokens: 163
  total_tokens: 164

